# Build a Crew to Tailor Job Applications V2

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [213]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [214]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [215]:
import os
# from utils import get_openai_api_key, get_serper_api_key

openai_api_key = "sk-proj-zLEsC4gv5zJ1EWsn3pgUT3BlbkFJrIDdyUFGu2GPzwjhVfbv"
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPER_API_KEY"] = "52a7b98fc031f9e0a3b5aba2ef667a9c5db81d17"

## crewAI Tools

In [216]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  DirectoryReadTool,
  DirectorySearchTool,
  PDFSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = DirectoryReadTool(file_path='./data')
search_resume = DirectorySearchTool(file_path='./data')
semantic_search_resume = PDFSearchTool()

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [217]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

### Candidate Profile Agents

In [218]:
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do incredible research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool,
             read_resume, search_resume, semantic_search_resume],
    verbose=True,
    allow_delegation=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resumes and cover letters."
    )
)

In [219]:
profile_enricher = Agent(
    role="Profile Enricher for Engineers",
    goal="Enriches candidate data with additional insights (e.g., skill proficiency, recent projects).",
    backstory=(
        "You are a master at enriching candidate data with additional insights. "
        "You can help candidates stand out by providing a more comprehensive view of their skills, "
        "proficiencies, and recent projects."
    ),
    tools = [scrape_tool,
             read_resume, search_resume, semantic_search_resume],
    verbose=True,
)

### Job Requirements Agents

In [220]:
job_researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool],
    verbose=True,
    allow_delegation=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application and cover letter tailoring."
    )
)

In [221]:
job_req_enrichment_agent = Agent(
    role="Job Requirement Enrichment Agent",
    goal="Expands on job requirements with industry standards, typical expectations, and common keywords.",
    tools = [search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "As a Job Requirement Enrichment Agent, "
        "you are equipped with the ability to "
        "expand on job requirements by "
        "providing industry standards, typical "
        "expectations, and common keywords."
    )
)

### Matching and analysis agent(s)

In [222]:
profile_matching_agent = Agent(
    role="Profile Matching Agent",
    goal="Matches the candidate’s skills and experiences with the job requirements.",
    verbose=True,
    backstory=(
        "As a Profile Matching Agent, you are "
        "tasked with matching the candidate’s "
        "skills and experiences with the job requirements."
    )
)

In [223]:
gap_analysis_agent = Agent(
    role="Gap Analysis Agent",
    goal="Identifies gaps between the candidate’s skills and the job requirements.",
    verbose=True,
    backstory=(
        "As a Gap Analysis Agent, you are tasked "
        "with identifying gaps between the candidate’s "
        "skills and the job requirements. "
    )
)

### Cover Letter Writing / Reviewing / QA

In [224]:
cover_letter_writer = Agent(
    role="Y-Combinator Cover Letter Writer for ML/AI Engineers",
    goal="Compose a cover letter for a Y-Combinator startup application",
    tools = [scrape_tool,
             read_resume, search_resume, semantic_search_resume],
    verbose=True,
    allow_delegation=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at composing cover letters for applicants to y-combinator startups. "
        "You understand what is important to startup founders "
        "(especially those in y-combinator) in the ML/AI space. "
        "You know how to highlight relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [225]:
cover_letter_reviewer = Agent(
    role="Y-Combinator startup ML/AI Engineering hiring manager",
    goal="Review cover letters for Y-Combinator startup applications, compare them to job requirements, provide feedback to the cover letter writer",
    tools = [scrape_tool],
    backstory=(
        "As a seasoned ML/AI Engineering Hiring Manager, you have dedicated your career to "
        "identifying and nurturing top talent in the tech industry. Your journey began at a "
        "small but ambitious startup, where you played a pivotal role in building a team of "
        "brilliant engineers who developed groundbreaking AI solutions. Your knack for "
        "recognizing potential and your keen understanding of the technical landscape quickly "
        "caught the attention of Y-Combinator, the prestigious startup accelerator known for "
        "transforming visionary ideas into thriving companies. "
        "With a deep-rooted passion for innovation, you joined Y-Combinator to lead the charge "
        "in recruiting exceptional ML/AI engineers for their rapidly growing portfolio of "
        "startups. Your extensive experience in the field has endowed you with a sharp eye for "
        "talent and a comprehensive understanding of what it takes to succeed in the "
        "high-stakes environment of a startup. "
        "Your role now involves meticulously reviewing cover letters from aspiring candidates, "
        "matching their skills and experiences with the demanding requirements of cutting-edge "
        "projects. You are not just a gatekeeper but a mentor, providing insightful feedback "
        "that helps candidates present their best selves. Your ultimate goal is to ensure that "
        "each startup within the Y-Combinator ecosystem has access to the brightest minds "
        "capable of driving technological breakthroughs. "
        "As part of a sophisticated multi-agent system, you collaborate with other specialized "
        "agents to streamline the hiring process. Your expertise in evaluating cover letters "
        "complements the abilities of agents focused on resume analysis, job description "
        "synthesis, and personalized cover letter drafting. Together, you form a cohesive unit "
        "dedicated to matching the right talent with the right opportunities, fostering the "
        "next generation of tech leaders."
    ),
    verbose=True,
    allow_delegation=True
)

In [226]:
qa_agent = Agent(
    role="Consistency Checker",
    goal="Ensure consistency in the information provided by the resume, cover letter, and job requirements",
    backstory=(
        "As a Consistency Checker, your role is to ensure that the information provided by the resume, "
        "cover letter, and job requirements is consistent and aligned. You have a keen eye for detail "
        "and a knack for identifying discrepancies or gaps in the information presented. Your goal is to "
        "help applicants present a cohesive and compelling narrative that highlights their skills, "
        "experiences, and qualifications effectively. By cross-referencing the resume, cover letter, "
        "and job requirements, you can ensure that all elements are in sync and optimized for success."
    ),
    verbose=True,
    allow_delegation=True
)

## Creating Tasks

### Candidate Profile Tasks

In [240]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed and comprehensive personal and professional profile "
        "using the candidate's resume ({resume_path}) and "
        "their portfolio files (located in {portfolio_dir}). "
        # "and LinkedIn ({linkedin_url}) profile"
        "The files in the portfolio directory {portfolio_dir} may include "
        "things like quarterly reviews from previous jobs or other relevant docs. Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, work experiences, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True,
    output_file="profile.md"
)

In [241]:
enrich_profile_task = Task(
    description=(
        "Enrich the candidate profiles provided by the profiler agent. "
        "Utilize the available tools to gather additional insights such as "
        "skill proficiency levels, recent projects, and other relevant information. "
        "Focus on making the profiles more comprehensive and impactful."
    ),
    expected_output=(
        "An enriched candidate profile that includes additional details about "
        "skills, proficiency levels, recent projects, and any other relevant insights. "
        "The enriched profile should be detailed and formatted as a comprehensive report."
    ),
    agent=profile_enricher,
    async_execution=True
)

### Job Requirements tasks

In [242]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences, as well as preferred "
        "skills qualifications and experiences."
    ),
    agent=job_researcher,
    async_execution=True
)

In [243]:
enrich_job_requirements_task = Task(
    description=(
        "Enrich the job requirements provided by the job researcher agent. "
        "Utilize the available tools to gather additional insights such as "
        "industry standards, typical expectations, and common keywords relevant to the job postings. "
        "Focus on making the job requirements more comprehensive and aligned with industry best practices."
    ),
    expected_output=(
        "An enriched set of job requirements that includes additional details about "
        "industry standards, typical expectations, and common keywords. "
        "The enriched requirements should be detailed and formatted as a comprehensive report."
    ),
    tools=[search_tool, scrape_tool],
    agent=job_req_enrichment_agent,
    async_execution=True
)

### Matching and analysis tasks

In [244]:
match_profile_task = Task(
    description=(
        "Match the candidate’s skills and experiences with the job requirements provided. "
        "Utilize the available data to create a comprehensive matching report that highlights "
        "how well the candidate fits the job requirements."
    ),
    expected_output=(
        "A detailed matching report that outlines the candidate’s skills and experiences "
        "in relation to the job requirements. The report should clearly indicate the areas "
        "where the candidate meets or exceeds the requirements."
    ),
    agent=profile_matching_agent,
    async_execution=True
)

In [245]:
analyze_gaps_task = Task(
    description=(
        "Identify gaps between the candidate’s skills and the job requirements provided. "
        "Utilize the available data to create a comprehensive gap analysis report that highlights "
        "the areas where the candidate’s skills do not fully meet the job requirements."
    ),
    expected_output=(
        "A detailed gap analysis report that outlines the candidate’s skills in relation to the job requirements. "
        "The report should clearly indicate the areas where the candidate’s skills are lacking or need improvement."
    ),
    agent=gap_analysis_agent,
    async_execution=True
)

### Cover Letter writing / reviewing / qa

In [246]:
# Task for Resume Strategist Agent: Align cover letter with Job Requirements
cover_letter_compose_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, write the cover letter to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "cover letter content. Make sure the cover letter is written"
        "specifically for the role described in the job posting ({job_posting_url})"
        "and includes only truthful information (i.e. do not make up information)."
    ),
    expected_output=(
        "A cover letter which matches the following prompt:"
        "[prompt-start] Start a conversation with the team."
        "Share something about you, what you're looking for,"
        "or why the company interests you. [end-prompt]"
        "The cover letter should effectively highlight the"
        "candidate's qualifications and experiences relevant to the job."
        "Be sure to keep the cover letter concise (less than 130 words)"
        "and to the point, highlighting only those skills and experiences"
        "which are most relevant to the job posting ({job_posting_url})."
        "Tailor the cover letter to the specific role and company."
        "Express why you are interested in the role and how your"
        "experience aligns with the job requirements."
    ),
    output_file="cover_letter.md",
    context=[research_task, profile_task],
    agent=cover_letter_writer
)


In [247]:
review_cover_letter_task = Task(
    description=(
        "Review cover letters for Y-Combinator startup applications. "
        "Compare them to the provided job requirements at ({job_posting_url}) and give detailed feedback to the cover letter writer. "
        "Focus on how well the candidate's skills and experiences match the job requirements. "
        "Provide suggestions for improving the cover letter to better align with the job description."
    ),
    expected_output=(
        "A comprehensive feedback report on the cover letter, highlighting strengths, weaknesses, and "
        "specific suggestions for improvement. The report should include a clear assessment of how well "
        "the cover letter matches the job requirements and any additional advice for making the application stand out."
    ),
    tools=[scrape_tool],
    agent=cover_letter_reviewer,
    async_execution=True,
)

In [248]:
check_consistency_task = Task(
    description=(
        "Ensure that the contents of the cover letter are factual and consistent with the job requirements and the candidate's resume. "
        "Cross-reference the details in each document to identify any discrepancies or gaps. "
        "Focus on ensuring that the information is aligned and presents a cohesive narrative. "
        "Highlight any inconsistencies found and suggest possible corrections."
    ),
    expected_output=(
        "A detailed consistency report that outlines any discrepancies or gaps found between the cover letter and "
        "other relevant documents (candidate's resume, candidate's linkedIn profile, the job requirements, etc). The report should provide suggestions for corrections to ensure "
        "the cover letter is aligned with job requirements and canditate documents, and presents a cohesive narrative."
    ),
    agent=qa_agent,
    async_execution=True
)

## Creating the Crew

In [249]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

In [250]:
job_application_crew = Crew(
    agents=[
            profiler,
            # profile_enricher,
            job_researcher,
            # job_req_enrichment_agent,
        #     profile_matching_agent,
        #     gap_analysis_agent,
            cover_letter_writer,
            cover_letter_reviewer,
            qa_agent
            ],

    tasks=[
            profile_task,
            # enrich_profile_task,
            research_task,
            # enrich_job_requirements_task,
        #     match_profile_task,
        #     analyze_gaps_task,
            cover_letter_compose_task,
            review_cover_letter_task,
            check_consistency_task
           ],
    manager_llm=ChatOpenAI(model="gpt-4-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

2024-05-28 18:17:27,592 - 8671874048 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [253]:
job_application_inputs = {
    'job_posting_url': 'https://www.workatastartup.com/jobs/66658',
    'resume_path': '../data/seth_resume.pdf',
    # 'github_url': 'https://github.com/sethcoast',
    # 'linkedin_url': 'https://www.linkedin.com/in/seth-donaldson/',
    'portfolio_dir': '../data',
    # 'personal_writeup': """Seth is an accomplished Machine Learning Engineer
    # with 2 years of experience in ML infrastructure engineering; 4 years 
    # of internship and research experience in deep learning for computer vision;
    # and 2 years of experience as a mobile app developer. He is proficient in
    # python, scala, and java, and has experience with a variety of machine
    # learning libraries and frameworks (including both PyTorch and TensorFlow).
    # Seth holds a Master's degree in Computer Science from Columbia University.
    # He is passionate about learning languages. He has learned Brazilian Portuguese,
    # and is currently learning Japanese. Seth is a quick learner and a team player. He is
    # looking for opportunities to apply his skills in a dynamic and innovative
    # startup environment."""
}

In [254]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Compile a detailed and comprehensive personal and professional profile using the candidate's resume (../data/seth_resume.pdf) and their portfolio files (located in ../data). The files in the portfolio directory ../data may include things like quarterly reviews from previous jobs or other relevant docs. Utilize tools to extract and synthesize information from these sources.
 [DEBUG]: [Crew Manager] Task output: None
 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Analyze the job posting URL provided (https://www.workatastartup.com/jobs/66658) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: [Crew Manager] Task output: None
 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Using the profile and job requirements obtained from previous tasks, write the cover letter to highlight the most relevant areas



> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...
Thought: First, I need to review the job description at the provided URL to understand the key qualifications and requirements the company is looking for in a candidate. This will allow me to accurately assess how well the cover letter aligns with these requirements and identify areas for improvement.

Action: Ask question to co-worker
Action Input: {"coworker": "Tech Job Researcher", "question": "Can you provide a detailed summary of the key qualifications and requirements listed for the AI/ML Engineer position at VectorShift?", "context": "The job description is found at the URL https://www.workatastartup.com/jobs/66658. I need to understand what specific skills, experiences, and characteristics the company is looking for in candidates to effectively review a cover letter and provide feedback."}

> Entering new CrewAgentExecutor chain...
I need to extract detailed information from the job posting 

- Dislplay the generated `tailored_resume.md` file.

In [256]:
from IPython.display import Markdown, display
display(Markdown("./cover_letter.md"))

Dear Hiring Team at VectorShift,

I am writing to express Seth's keen interest in the AI/ML Engineer position at VectorShift. With a background in robotics projects, active participation in STEM discussions, and a commitment to diversity in computing, Seth is well-equipped to excel in this role. His proficiency in Python, experience with AI and machine learning, and familiarity with building RAG workflows make him a perfect fit for the position. Seth's leadership potential, ability to work directly with founders, lead technical decisions, and stay updated with AI developments, set him apart as a valuable asset to your team. Thank you for considering Seth for this opportunity.

Warm regards,
[Your Name]

- Dislplay the generated `interview_materials.md` file.

In [ ]:
display(Markdown("./interview_materials.md"))